<a href="https://colab.research.google.com/github/snanditachn/Nandita.S_cts/blob/main/Nandita_S_2211551_Case_Study_3_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting zomato ratings using NLP and Random forest regressor.  

Importing the required libraries and packages

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Importing the dataset. Zomato reviews

In [ ]:
from google.colab import files
 
 
uploaded = files.upload()

Saving Zomato_reviews.csv to Zomato_reviews (1).csv


In [ ]:
reviews0 = pd.read_csv("Zomato_reviews.csv", encoding= 'unicode_escape')

In [ ]:
reviews0.head()

,rating,review_text
0,1.0,"Their service is worst, pricing in menu is dif..."
1,5.0,really appreciate their quality and timing . I...
2,4.0,"Went there on a Friday night, the place was su..."
3,4.0,A very decent place serving good food.\r\nOrde...
4,5.0,One of the BEST places for steaks in the city....


getting the uniques ratings available in the dataset

In [ ]:
reviews0.rating.unique()

array([1. , 5. , 4. , 3. , 2. , 4.5, 1.5, 3.5, 2.5])

we have 5 point rating scale, but some ratings also include 1.5, 2.5, 3.5, 4.5 as ratings.

In [ ]:
reviews0.describe(include="all")

,rating,review_text
count,27762.000000,27748
unique,NaN,10548
top,NaN,good
freq,NaN,278
mean,3.665784,NaN
std,1.284573,NaN
min,1.000000,NaN
25%,3.000000,NaN
50%,4.000000,NaN
75%,5.000000,NaN


droping the null values in review text columns by creating a duplicate copy of the dataframe with name reviews1

In [ ]:
reviews1 = reviews0[~reviews0.review_text.isnull()].copy()
reviews1.reset_index(inplace=True, drop=True)

printing shape of the original dataframe and the null values dropped dataframe seperately

In [ ]:
reviews0.shape, reviews1.shape

((27762, 2), (27748, 2))

## Converting to list for easy manipulation

creating a reviews_list dataframe with the review text column values, and calculate the length of that list.

In [ ]:
reviews_list = reviews1.review_text.values

In [ ]:
len(reviews_list)

27748

covert all the review text to lower case and add it to reviews_lower

In [ ]:
reviews_lower = [txt.lower() for txt in reviews_list]

reading the 2nd and 3rd review text in the reviews_lower df

In [ ]:
reviews_lower[2:4]

['went there on a friday night, the place was surprisingly empty. interesting menu which is almost fully made of dosas. i had bullseye dosa and cheese masala dosa. the bullseye dosa was really good, with the egg perfectly cooked to a half boiled state. the masala in the cheese masala was good, but the cheese was a bit too chewy for my liking. the chutney was good, the sambar was average. the dishes are reasonably priced.',
 'a very decent place serving good food.\r\nordered chilli fish, chicken & pork sizzler.\r\neverything tasted good but pork could have been slightly better cooked.\r\ntried 2 beverages, both were very sweet.']

splitting the reviews_lower by space and join it again and display.

In [ ]:
reviews_lower = [" ".join(txt.split()) for txt in reviews_lower]

In [ ]:
reviews_lower[2:4]

['went there on a friday night, the place was surprisingly empty. interesting menu which is almost fully made of dosas. i had bullseye dosa and cheese masala dosa. the bullseye dosa was really good, with the egg perfectly cooked to a half boiled state. the masala in the cheese masala was good, but the cheese was a bit too chewy for my liking. the chutney was good, the sambar was average. the dishes are reasonably priced.',
 'a very decent place serving good food. ordered chilli fish, chicken & pork sizzler. everything tasted good but pork could have been slightly better cooked. tried 2 beverages, both were very sweet.']

## Tokenize

Word tokenization is the process of splitting a large sample of text into words. This is a requirement in natural language processing tasks where each word needs to be captured and subjected to further analysis like classifying and counting them for a particular sentiment etc.

In [ ]:
from nltk.tokenize import word_tokenize

tokenizing the 1st review in reviews_lower

In [ ]:
print(word_tokenize(reviews_lower[0]))

['their', 'service', 'is', 'worst', ',', 'pricing', 'in', 'menu', 'is', 'different', 'from', 'bill', '.', 'they', 'can', 'give', 'you', 'a', 'bill', 'with', 'increased', 'pricing', '.', 'even', 'for', 'serving', 'water', ',', 'menu', ',', 'order', 'you', 'need', 'to', 'call', 'them', '3-4', 'times', 'even', 'on', 'a', 'non', 'busy', 'day', '.']


tokenizing the whole df reviews_lowerand displaying the first tokenized review

In [ ]:
reviews_tokens = [word_tokenize(sent) for sent in reviews_lower]
print(reviews_tokens[0])

['their', 'service', 'is', 'worst', ',', 'pricing', 'in', 'menu', 'is', 'different', 'from', 'bill', '.', 'they', 'can', 'give', 'you', 'a', 'bill', 'with', 'increased', 'pricing', '.', 'even', 'for', 'serving', 'water', ',', 'menu', ',', 'order', 'you', 'need', 'to', 'call', 'them', '3-4', 'times', 'even', 'on', 'a', 'non', 'busy', 'day', '.']


## Remove stop words and punctuations

A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine has been programmed to ignore, both when indexing entries for searching and when retrieving them as the result of a search query. 

In order to remove stopwords and punctuation using NLTK, we have to download all the stop words using nltk. download('stopwords'), then we have to specify the language for which we want to remove the stopwords, therefore, we use stopwords. words('english') to specify and save it to the variable.

In [ ]:
from nltk.corpus import stopwords
from string import punctuation
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_nltk = stopwords.words("english")
stop_punct = list(punctuation)

printing all the stop words in the dataset

In [ ]:
print(stop_nltk)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

remove 4 stop words found,namely no, not, don, won

In [ ]:
stop_nltk.remove("no")
stop_nltk.remove("not")
stop_nltk.remove("don")
stop_nltk.remove("won")

checking whether they are removed

In [ ]:
"no" in stop_nltk

False

creating stop_final with all stop words, stop punctuations and some of the prefered stop words except the words found by nltk 

In [ ]:
stop_final = stop_nltk + stop_punct + ["...", "``","''", "====", "must"]


creating a function del_stop with sent variable, to return the terms not in stop_final but in the sent variable.

In [ ]:
def del_stop(sent):
    return [term for term in sent if term not in stop_final]

calling the function del_stop with the second review in reviews_tokens(tokenized reviews)

In [ ]:
del_stop(reviews_tokens[1])

['really',
 'appreciate',
 'quality',
 'timing',
 'tried',
 'thattil',
 'kutti',
 'dosa',
 "'ve",
 'addicted',
 'dosa',
 'really',
 'chutney',
 'really',
 'good',
 'money',
 'worth',
 'much',
 'better',
 'thattukada',
 'try']

use the del_stop function to the whole df reviews_tokens

In [ ]:
reviews_clean = [del_stop(sent) for sent in reviews_tokens]

joining the reviews_clean df and displaying first 2 reviews

In [ ]:
reviews_clean = [" ".join(sent) for sent in reviews_clean]
reviews_clean[:2]

['service worst pricing menu different bill give bill increased pricing even serving water menu order need call 3-4 times even non busy day',
 "really appreciate quality timing tried thattil kutti dosa 've addicted dosa really chutney really good money worth much better thattukada try"]

## Separate X and Y and perform train test split, 70-30

In [ ]:
len(reviews_clean)

27748

the independent variable is X (reviews_clean) and dependent varaible is y (reviews1.rating)

In [ ]:
X = reviews_clean
y = reviews1.rating

spliting the train and test for indep and dep variable

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)


## Document term matrix using Tf-Idf

NLP uses tf-idf technique to convert text documents to a machine understandable form. Each sentence is a document and words in the sentence are tokens. Tfidf vectorizer creates a matrix with documents and token scores therefore it is also known as document term matrix (dtm)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Vectorizer is used to convert a collection of raw documents to a matrix of TF-IDF features.
here we have limited the maximum number of feature to 5000, if we are not mentioning any value it will default to the maximum number of elements in our dataset. 

In [ ]:
vectorizer = TfidfVectorizer(max_features = 5000)

In [ ]:
len(X_train), len(X_test)

(19423, 8325)

fit and transform training set

In [ ]:
X_train_bow = vectorizer.fit_transform(X_train)

transform the test set

In [ ]:
X_test_bow = vectorizer.transform(X_test)

shape of the train and test set

In [ ]:
X_train_bow.shape, X_test_bow.shape

((19423, 5000), (8325, 5000))

## Model building

Random forest is a Supervised Machine Learning Algorithm that is used widely in Classification and Regression problems. It builds decision trees on different samples and takes their majority vote for classification and average in case of regression.

A random forest regressor is a meta estimator that fits a number of classifying decision trees on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


In [ ]:
?RandomForestRegressor

In [ ]:
learner_rf = RandomForestRegressor(random_state=42)

In [ ]:
learner_rf.fit(X_train_bow, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
y_train_preds = learner_rf.predict(X_train_bow)

In [ ]:
from sklearn.metrics import mean_squared_error


Based on a rule of thumb, it can be said that RMSE values between 0.2 and 0.5 shows that the model can relatively predict the data accurately.

In [ ]:
mean_squared_error(y_train, y_train_preds)**0.5

0.23684233164605095

RMSE is good, still we can improve the score.
We have many methods for this namely,



### Features which make predictions of the model better

There are primarily 3 features which can be tuned to improve the predictive power of the model :

1.a. **max_features**:
These are the maximum number of features Random Forest is allowed to try in individual tree. There are multiple options available in Python to assign maximum features. Here are a few of them :

Auto/None : This will simply take all the features which make sense in every tree.Here we simply do not put any restrictions on the individual tree.

sqrt : This option will take square root of the total number of features in individual run. For instance, if the total number of variables are 100, we can only take 10 of them in individual tree.”log2″ is another similar type of option for max_features.

0.2 : This option allows the random forest to take 20% of variables in individual run. We can assign and value in a format “0.x” where we want x% of features to be considered.

Increasing max_features generally improves the performance of the model as at each node now we have a higher number of options to be considered. However, this is not necessarily true as this decreases the diversity of individual tree which is the USP of random forest. But, for sure, you decrease the speed of algorithm by increasing the max_features. Hence, you need to strike the right balance and choose the optimal max_features.

1.b. **n_estimators** :
This is the number of trees you want to build before taking the maximum voting or averages of predictions. Higher number of trees give you better performance but makes your code slower. You should choose as high value as your processor can handle because this makes your predictions stronger and more stable.

1.c. **min_sample_leaf** :
 Leaf is the end node of a decision tree. A smaller leaf makes the model more prone to capturing noise in train data. Generally preferable is a minimum leaf size of more than 50. However, you should try multiple leaf sizes to find the most optimum for your use case.

### Features which will make the model training easier

There are a few attributes which have a direct impact on model training speed. Following are the key parameters which you can tune for model speed :

2.a. **n_jobs **:
This parameter tells the engine how many processors is it allowed to use. A value of “-1” means there is no restriction whereas a value of “1” means it can only use one processor.

2.b. **random_state** :
This parameter makes a solution easy to replicate. A definite value of random_state will always produce same results if given with same parameters and training data. I have personally found an ensemble with multiple models of different random states and all optimum parameters sometime performs better than individual random state.

2.c. **oob_score** :
This is a random forest cross validation method. It is very similar to leave one out validation technique, however, this is so much faster. This method simply tags every observation used in different tress. And then it finds out a maximum vote score for every observation based on only trees which did not use this particular observation to train itself.

In [ ]:
# max_features adjusting
learner_rf = RandomForestRegressor(random_state=42, max_features = "sqrt")

In [ ]:
learner_rf.fit(X_train_bow, y_train)

RandomForestRegressor(max_features='sqrt', random_state=42)

In [ ]:
y_train_preds = learner_rf.predict(X_train_bow)

In [ ]:
mean_squared_error(y_train, y_train_preds)**0.5

0.2314372225647542

the RMSE has improved slightly while adjusting max_features

In [ ]:
# n_estimators adjusting
learner_rf = RandomForestRegressor(random_state=42, n_estimators = 200 )

In [ ]:
learner_rf.fit(X_train_bow, y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [ ]:
y_train_preds = learner_rf.predict(X_train_bow)

In [ ]:
mean_squared_error(y_train, y_train_preds)**0.5

0.23536129985984264

the RMSE is improved slightly

In [ ]:
# min_sample_leaf adjusting
learner_rf = RandomForestRegressor(random_state=42, min_samples_leaf=50 )

In [ ]:
learner_rf.fit(X_train_bow, y_train)

RandomForestRegressor(min_samples_leaf=50, random_state=42)

In [ ]:
y_train_preds = learner_rf.predict(X_train_bow)

In [ ]:
mean_squared_error(y_train, y_train_preds)**0.5

0.8009312023055373

the RMSE has increased drastically indiacating overfit of the model.

In [ ]:
# n_jobs adjusting
learner_rf = RandomForestRegressor(random_state=42, n_jobs = -1)

In [ ]:
learner_rf.fit(X_train_bow, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [ ]:
y_train_preds = learner_rf.predict(X_train_bow)

In [ ]:
mean_squared_error(y_train, y_train_preds)**0.5

0.23684233164605095

RMSE has not improved 

In [ ]:
# random_state adjusting
learner_rf = RandomForestRegressor(random_state=50)

In [ ]:
learner_rf.fit(X_train_bow, y_train)

RandomForestRegressor(random_state=50)

In [ ]:
y_train_preds = learner_rf.predict(X_train_bow)

In [ ]:
mean_squared_error(y_train, y_train_preds)**0.5

0.23707204411829896

RMSE is not improving with random state adjustment also

In [ ]:
# oob_score adjusting
learner_rf = RandomForestRegressor(random_state=1, oob_score = True )

In [ ]:
learner_rf.fit(X_train_bow, y_train)

RandomForestRegressor(oob_score=True, random_state=1)

In [ ]:
y_train_preds = learner_rf.predict(X_train_bow)

In [ ]:
mean_squared_error(y_train, y_train_preds)**0.5

0.23658504429975258

RMSE is still not improving

## Increasing the number of trees

In [ ]:
learner_rf = RandomForestRegressor(random_state=42, n_estimators=30)


In [ ]:
%%time
learner_rf.fit(X_train_bow, y_train)

CPU times: user 2min 21s, sys: 175 ms, total: 2min 21s
Wall time: 2min 24s


RandomForestRegressor(n_estimators=30, random_state=42)

In [ ]:
y_train_preds = learner_rf.predict(X_train_bow)

In [ ]:
mean_squared_error(y_train, y_train_preds)**0.5

0.24670225379076136

## Hyper-parameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
?RandomForestRegressor

In [ ]:
learner_rf = RandomForestRegressor(random_state=42)

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_features': [500, "sqrt", "log2", "auto"],
    'max_depth': [10, 15, 20]
}

In [ ]:
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = learner_rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 1, scoring = "neg_mean_squared_error" )


In [ ]:
grid_search.fit(X_train_bow, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [10, 15, 20],
                         'max_features': [500, 'sqrt', 'log2', 'auto']},
             scoring='neg_mean_squared_error', verbose=1)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_search.cv_results_

{'mean_fit_time': array([ 10.71263108,   1.99054923,   0.73964562,  60.42418742,
         14.12563343,   2.11304016,   0.83744516, 101.09650741,
         22.7613193 ,   3.50108776,   1.10733013, 134.95551219]),
 'std_fit_time': array([1.08661467e+00, 4.41090736e-01, 4.90119950e-02, 1.48677498e+00,
        6.14724931e-01, 2.98468634e-02, 8.47460514e-03, 1.36583531e+00,
        9.62227674e-01, 5.63908708e-01, 1.25020400e-02, 2.04455173e+01]),
 'mean_score_time': array([0.1310946 , 0.11562757, 0.09531479, 0.0831768 , 0.10245252,
        0.08770051, 0.08230524, 0.09840479, 0.11253448, 0.10615892,
        0.09285569, 0.10584531]),
 'std_score_time': array([0.04981019, 0.0355047 , 0.02865092, 0.0032912 , 0.01171679,
        0.00183926, 0.00163911, 0.00258491, 0.00333736, 0.00718951,
        0.00181719, 0.01201283]),
 'param_max_depth': masked_array(data=[10, 10, 10, 10, 15, 15, 15, 15, 20, 20, 20, 20],
              mask=[False, False, False, False, False, False, False, False,
              

In [ ]:
grid_search.best_estimator_

RandomForestRegressor(max_depth=20, max_features=500, random_state=42)

In [ ]:
y_train_pred = grid_search.best_estimator_.predict(X_train_bow)

In [ ]:
y_test_pred = grid_search.best_estimator_.predict(X_test_bow)


In [ ]:
mean_squared_error(y_train, y_train_pred)**0.5

0.6625331294952135

In [ ]:
mean_squared_error(y_test, y_test_pred)**0.5

0.7272534467721806

## Identifying mismatch cases

In [ ]:
res_df = pd.DataFrame({'review':X_test, 'rating':y_test, 'rating_pred':y_test_pred})

In [ ]:
res_df[(res_df.rating - res_df.rating_pred)>=2].shape

(6, 3)

In [ ]:
res_df[(res_df.rating - res_df.rating_pred)>=2]

,review,rating,rating_pred
7277,life saviours serving excellent food worst tim...,5.0,2.150063
1818,value money ordered second time,5.0,2.964410
4771,not good,5.0,2.189819
16510,may not polished serving packaging etc never b...,5.0,1.840416
14845,oh memories place first drink bangalore almost...,5.0,2.618844
16515,may not polished serving packaging etc never b...,5.0,1.840416
